# Urban Green Space Analysis and Optimization Tool

In [2]:
!pip install osmnx geopandas folium matplotlib scikit-learn

In [3]:
pip install osmnx


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install folium

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [5]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
from shapely.geometry import Polygon

In [7]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import folium
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


In [8]:
# Function to fetch urban boundary and green area data
def fetch_urban_data(city_name):
    import osmnx as ox
    print(f"Fetching data for {city_name}...")
    
    # Fetch urban boundary
    boundary = ox.geocode_to_gdf(city_name)
    
    # Fetch all types of green areas suitable for planting trees
    green_tags = {
        "leisure": ["park", "garden"],
        "landuse": ["forest", "grass", "meadow"],
        "natural": ["wood", "grassland"]
    }
    green_areas = ox.features_from_place(city_name, tags=green_tags)
    
    return boundary, green_areas

# Example: Fetch data for a city
city_name = "Mysuru, India"
boundary, green_areas = fetch_urban_data(city_name)

# Display data
print(boundary.head())
print(green_areas.head())


Fetching data for Mysuru, India...
                                            geometry  bbox_west  bbox_south  \
0  POLYGON ((75.90868 12.39816, 75.90896 12.39721...   75.90868    11.73801   

   bbox_east  bbox_north   place_id  osm_type   osm_id        lat        lon  \
0  77.134284   12.655683  233291732  relation  2020510  12.196832  76.612047   

      class            type  place_rank  importance     addresstype    name  \
0  boundary  administrative          10     0.51349  state_district  Mysuru   

               display_name  
0  Mysuru, Karnataka, India  
                                     geometry  \
element id                                      
node    2621900848  POINT (76.38829 12.21265)   
        4012599895  POINT (76.63293 12.30178)   
        8984979753  POINT (76.62513 12.29959)   
        8991839276  POINT (76.62501 12.29276)   
        9340717203  POINT (76.58452 12.25077)   

                                                                 name natural  \
e

In [9]:
# Function to calculate green space coverage
def analyze_green_space(boundary, green_areas):
    green_areas['area_sqm'] = green_areas.geometry.area
    total_green_space = green_areas['area_sqm'].sum()
    boundary_area = boundary.geometry.area.iloc[0]
    coverage_percentage = (total_green_space / boundary_area) * 100
    
    print(f"Total Green Space: {total_green_space:.2f} sq.m")
    print(f"City Area: {boundary_area:.2f} sq.m")
    print(f"Green Space Coverage: {coverage_percentage:.2f}%")
    
    return total_green_space, coverage_percentage

# Analyze green space for the city
total_green_space, coverage_percentage = analyze_green_space(boundary, green_areas)


Total Green Space: 0.36 sq.m
City Area: 0.52 sq.m
Green Space Coverage: 68.65%


C:\Users\Google\AppData\Local\Temp\ipykernel_150240\1591048491.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  green_areas['area_sqm'] = green_areas.geometry.area
C:\Users\Google\AppData\Local\Temp\ipykernel_150240\1591048491.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  boundary_area = boundary.geometry.area.iloc[0]


In [10]:
# Function to predict environmental impact based on green space area
def predict_environmental_impact(green_space_areas, temperature_reductions):
    model = LinearRegression()
    green_space_areas = green_space_areas.values.reshape(-1, 1)
    temperature_reductions = temperature_reductions.values
    model.fit(green_space_areas, temperature_reductions)
    
    # Predict impact for 1000 sq.m of green space
    predicted_reduction = model.predict([[1000]])[0]
    print(f"Predicted Temperature Reduction for 1000 sq.m Green Space: {predicted_reduction:.2f}°C")
    return model

# Example data for prediction
green_space_data = pd.Series([100, 500, 1000, 1500, 2000])  # Green space areas in sq.m
temp_reduction_data = pd.Series([0.1, 0.5, 1.0, 1.5, 2.0])  # Corresponding temperature reductions in °C

# Train the model and predict
model = predict_environmental_impact(green_space_data, temp_reduction_data)


Predicted Temperature Reduction for 1000 sq.m Green Space: 1.00°C


In [11]:
from shapely.geometry import Polygon, MultiPolygon

# Function to create a map visualization
def visualize_data(boundary, green_areas, city_name):
    # Create a Folium map centered on the city's boundary centroid
    m = folium.Map(
        location=[boundary.geometry.centroid.y.iloc[0], boundary.geometry.centroid.x.iloc[0]], 
        zoom_start=12
    )
    
    # Add green areas to the map
    for _, row in green_areas.iterrows():
        geometry = row['geometry']
        
        # Check if the geometry is a Polygon
        if isinstance(geometry, Polygon):
            folium.Polygon(
                locations=[(pt[1], pt[0]) for pt in geometry.exterior.coords],
                color='green',
                fill=True,
                fill_opacity=0.5
            ).add_to(m)
        
        # Check if the geometry is a MultiPolygon
        elif isinstance(geometry, MultiPolygon):
            for polygon in geometry.geoms:  # Use .geoms to iterate over individual polygons
                folium.Polygon(
                    locations=[(pt[1], pt[0]) for pt in polygon.exterior.coords],
                    color='green',
                    fill=True,
                    fill_opacity=0.5
                ).add_to(m)
    
    # Save the map as an HTML file
    map_file = f"{city_name.replace(' ', '_')}_green_space_map.html"
    m.save(map_file)
    print(f"Map saved as {map_file}")
    return m

# Visualize the city's green spaces
map_result = visualize_data(boundary, green_areas, city_name)


C:\Users\Google\AppData\Local\Temp\ipykernel_150240\2444867427.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location=[boundary.geometry.centroid.y.iloc[0], boundary.geometry.centroid.x.iloc[0]],


Map saved as Mysuru,_India_green_space_map.html


In [12]:
# Display the map within Jupyter Notebook
from IPython.display import IFrame

map_file = f"{city_name.replace(' ', '_')}_green_space_map.html"
IFrame(map_file, width=800, height=600)
